In [198]:
import numpy as np
import ioh
import math
import random

In [199]:
ioh.ProblemClass.GRAPH.problems

{2000: 'MaxCut2000',
 2001: 'MaxCut2001',
 2002: 'MaxCut2002',
 2003: 'MaxCut2003',
 2004: 'MaxCut2004',
 2100: 'MaxCoverage2100',
 2101: 'MaxCoverage2101',
 2102: 'MaxCoverage2102',
 2103: 'MaxCoverage2103',
 2104: 'MaxCoverage2104',
 2105: 'MaxCoverage2105',
 2106: 'MaxCoverage2106',
 2107: 'MaxCoverage2107',
 2108: 'MaxCoverage2108',
 2109: 'MaxCoverage2109',
 2110: 'MaxCoverage2110',
 2111: 'MaxCoverage2111',
 2112: 'MaxCoverage2112',
 2113: 'MaxCoverage2113',
 2114: 'MaxCoverage2114',
 2115: 'MaxCoverage2115',
 2116: 'MaxCoverage2116',
 2117: 'MaxCoverage2117',
 2118: 'MaxCoverage2118',
 2119: 'MaxCoverage2119',
 2120: 'MaxCoverage2120',
 2121: 'MaxCoverage2121',
 2122: 'MaxCoverage2122',
 2123: 'MaxCoverage2123',
 2124: 'MaxCoverage2124',
 2125: 'MaxCoverage2125',
 2126: 'MaxCoverage2126',
 2127: 'MaxCoverage2127',
 2200: 'MaxInfluence2200',
 2201: 'MaxInfluence2201',
 2202: 'MaxInfluence2202',
 2203: 'MaxInfluence2203',
 2204: 'MaxInfluence2204',
 2205: 'MaxInfluence2205',
 2206

In [4]:
max_cut = ioh.get_problem(2000, problem_class=ioh.ProblemClass.GRAPH)
max_coverage = ioh.get_problem(2100, problem_class=ioh.ProblemClass.GRAPH)

In [5]:
print(max_cut.meta_data)
print(max_coverage.meta_data)

<MetaData: MaxCut2000 id: 2000 iid: 1 dim: 800>
<MetaData: MaxCoverage2100 id: 2100 iid: 1 dim: 450>


In [6]:
nb_it = 10000

# Multi-objective optimization

In [13]:
import random
from typing import Callable

In [14]:
class POSS:
    
    def __init__(self, n: int, pflip: int, b: int, I: Callable):
        self.n: int = n
        self.pflip = pflip #proba flit un bit
        self.b: int = b #budget
        self.I: Callable = I #isolation function : two solutions are comparable if they have the same I value
        
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:

        def constraint1(s):
            nb1 = np.count_nonzero(s[0])
            if nb1 >= 2*self.b: return -100000 # -INFINI
            return s[1]
        
        def constraint2(s):
            return -np.count_nonzero(s[0])
        
        def leq(x,y): # <=
            if constraint1(x) <= constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
                    
        def lt(x,y):  # <
            if constraint1(x) < constraint1(y):
                if constraint2(x) <= constraint2(y):
                    return True
            if constraint1(x) <= constraint1(y):
                if constraint2(x) < constraint2(y):
                    return True
        
        def bitwise_mutation(s):
            x = s[0].copy()

            for i in range(problem.meta_data.n_variables):
                flip = 1 if random.random() < self.pflip else 0
                if flip:
                    if x[i]: x[i] = 0
                    else: x[i] = 1
            
            return (x,problem(x))
        
        s0 = np.zeros(problem.meta_data.n_variables).astype(int)
        s = (s0,problem(s0))
        P = [(s[0].copy(), s[1])]
        for t in range(self.n):
            if not t%100:
                print("Iteration : ", t)
            s = P[np.random.randint(len(P))] #Select a solution from P uar            
            ns = bitwise_mutation(s)
            
            better_than_ns = [z for z in P if self.I(z[0])==self.I(ns[0]) and lt(ns,z)]

            if not better_than_ns :
                Q = [z for z in P if self.I(z)==self.I(ns) and leq(z,ns)]
                P = [z for z in P if not any(np.array_equal(z, q) for q in Q)]
                P.append((ns[0].copy(),ns[1]))

In [15]:
import math
max_coverage.reset()
dim_pb = max_coverage.meta_data.n_variables
poss = POSS(1000, 1/dim_pb, 10, lambda x: np.count_nonzero(x[0]))
poss(max_coverage)

Iteration :  0
Iteration :  100


/home/buzetq/.local/lib/python3.8/site-packages/numpy/core/numeric.py:2457: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a1, a2 = asarray(a1), asarray(a2)


Iteration :  200
Iteration :  300
Iteration :  400
Iteration :  500
Iteration :  600
Iteration :  700
Iteration :  800
Iteration :  900


In [16]:
max_coverage.state

<State evaluations: 1001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Algorithme par croissance

In [7]:
def flip(x,l):
    positions = np.random.choice(np.arange(0,len(x)), l, replace=False)
    y = x.copy()
    y[positions] = 1 - y[positions]
    return y

In [284]:
class Test:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n #nb d'itérations
        self.s: int = s #population size
        self.k: int = k #taux de croissance
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        x = np.zeros(dim).astype(int)
        y = problem(x)
        
        #Optimization
        for t in range(self.n-1):
            #On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = flip(x,self.k)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
            else:
                self.k = max(int(self.k*3/4),1)

In [285]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test = Test(nb_it//10, 10, 25)
test(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,

In [286]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test = Test(nb_it//1000, 1000, 5)
test(max_coverage)
max_coverage.state

<State evaluations: 9001 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

# Algorithme par décroissance

In [281]:
class Test3:
    
    def __init__(self, n: int, s: int, k:int):
        self.n: int = n #nb d'itérations
        self.s: int = s #population size
        self.k: int = k #taux de croissance
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        x = np.ones(dim).astype(int)
        y = problem(x)
        
        #Optimization
        for t in range(self.n-1):
            #On tire k échantillons et on garde celui qui fait le plus croitre la fonction
            pop = []
            for i in range(self.s):
                xx = flip(x,self.k)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if random.random() < self.s/dim or pop[self.s-1][1] > y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]
                self.k = min(int(self.k*4/3),dim)
            else:
                self.k = max(int(self.k*3/4),1)

In [282]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test3 = Test3(nb_it//10, 10, 25)
test3(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,

In [283]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test3 = Test3(nb_it//1000, 1000, 5)
test3(max_coverage)
max_coverage.state

<State evaluations: 9001 optimum_found: false current_best: <Solution x: [0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Combinaison d'algorithmes

In [261]:
def positive_binomial(n,p,nb_samples):
    gen = []
    for i in range(nb_samples):
        trial = np.random.binomial(n, p, 1)[0]
        while trial == 0:
            trial = np.random.binomial(n, p, 1)[0]
        gen.append(trial)
    return gen

In [256]:
def UMDA(problem,n,s):
    dim = problem.meta_data.n_variables
    
    def sample(p):
        x = np.zeros(dim,int)
        for j in range(dim):
            x[j] = 1 if random.random() < p[j] else 0
        y = problem(x)
        return (x,y)
    
    #Initialization
    z = []
    for i in range(s):
        x = np.random.randint(0,2,dim)
        y = problem(x)
        z.append((x,y))
    z.sort(key=lambda x: x[1])
    P = np.array(z[s-s//2:])
    
    #Optimization
    p = np.zeros(dim)
    for t in range(1,n):
        for j in range(dim):
            count = 0
            for k in range(s//2):
                if P[k,0][j] == 1: count +=1
            
            p[j] = 2*count/s
            if p[j] < 1/dim: p[j] = 1/dim
            if p[j] > 1 - 1/dim: p[j] = 1 - 1/dim
    
        for i in range(s):
            z[i] = sample(p)
        z.sort(key=lambda x: x[1])
        P = np.array(z[s-s//2:])
        
    return P

In [259]:
class ULog: #UMDA + EAlog

    def __init__(self,n,lamb):
        self.n = n
        self.lamb = lamb

    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        #Initialization
        U = UMDA(problem,10,5)
        x = U[len(U)-1][0]
        y = U[len(U)-1][1]
        p = 0.2
        
        #Optimization
        for t in range(self.n-1):
            pop = []
            for i in range(self.lamb):
                sample = np.random.normal(0,1)
                pp = 1/(1 + (1-p)/p * math.exp(0.22 * sample))
                l = positive_binomial(dim,pp,1)[0]
                xx = flip(x,l)
                yy = problem(xx)
                pop.append((xx,yy,pp))
                
            pop.sort(key=lambda x: x[1])
            p = pop[self.lamb-1][2]
            if pop[self.lamb-1][1] >= y:
                x = pop[self.lamb-1][0]
                y = pop[self.lamb-1][1]

In [263]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
ulog = ULog(nb_it//10, 10)
ulog(max_coverage)
max_coverage.state

/tmp/ipykernel_15343/782673433.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])
/tmp/ipykernel_15343/782673433.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])


<State evaluations: 10040 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [266]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
ulog = ULog(nb_it//10, 10)
ulog(max_cut)
max_cut.state

/tmp/ipykernel_15343/782673433.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])
/tmp/ipykernel_15343/782673433.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  P = np.array(z[s-s//2:])


<State evaluations: 10040 optimum_found: false current_best: <Solution x: [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1

# Crossover par Moyenne

In [209]:
def gen_budget(b,N):
    pop = []
    for i in range(N):
        x = np.zeros(dim).astype(int)
        iflips = np.random.randint(0,dim,b)
        x[iflips] = 1
        y = problem(x)
        z = (x,y)
        pop.append(z)
    return pop

In [243]:
class Test2:
    def __init__(self, n: int, s: int, p: float):
        self.n: int = n #Number of iterations
        self.s: int = s #Population size
        self.p: float = p #Proba bitflip
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables
        
        def mutation(x):
            y = x.copy()
            for i in range(dim):
                y[i] = 1 - y[i] if random.random() < self.p else y[i]
            return y
            
        def crossover(pop):
            #Calcul de la distribution de probabilités 
            dist = np.zeros(dim)
            normalization = 0
            for i in range(self.s):
                dist = dist + pop[i][0] * pop[i][1]
                normalization += pop[i][1]
            dist = dist / normalization
            #Génération de la nouvelle population
            npop = []
            for t in range(self.s):
                x = np.zeros(dim).astype(int)
                for i in range(dim):
                    x[i] = 1 if random.random() < dist[i] else 0
                y = problem(x)
                z = (x,y)
                npop.append(z)
            return npop
        
        #Initialization
        x1 = np.zeros(dim).astype(int)
        y1 = problem(x1)
        x2 = np.random.randint(0,2,dim)
        y2 = problem(x2)
        x = x1 if y1 >= y2 else x2
        y = y1 if y1 >= y2 else y2

        #Optimization
        for t in range(self.n-2):
            pop = []
            for i in range(self.s):
                xx = mutation(x)
                yy = problem(xx)
                pop.append((xx,yy))
                
            pop = crossover(pop)
            
            pop.sort(key=lambda x: x[1])
            if pop[self.s-1][1] >= y:
                x = pop[self.s-1][0]
                y = pop[self.s-1][1]


In [244]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
test2 = Test2(nb_it//(2*10), 10, 1/dim)
test2(max_cut)

In [245]:
max_cut.state

<State evaluations: 9962 optimum_found: false current_best: <Solution x: [1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,

In [228]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
test2 = Test2(nb_it//(2*10), 10, 1/dim)
test2(max_coverage)

In [229]:
max_coverage.state

<State evaluations: 9962 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [ ]:
import math
max_cut.reset()
dim = max_cut.meta_data.n_variables
test2 = Test2(1000, 10, 1/dim, 400)
test2(max_cut)

In [ ]:
max_cut.state

# Big jump

In [289]:
np.random.randint(0, 10,2)

array([0, 1])

In [291]:
-1%4

3

In [312]:
def select_distant_indices(array_length, nb, min_dist, proba_pursue):
    selected_indices = [np.random.randint(0, array_length)]

    while len(selected_indices) < nb :
        new_index = np.random.randint(0, array_length)
        ok = True
        for index in selected_indices:
            d1 = abs(new_index - index)
            d2 = (-abs(new_index - index))%array_length
            if d1 <= min_dist or d2 <= min_dist:
                if random.random() > proba_pursue:
                    ok = False
        if ok: selected_indices.append(new_index)

    return selected_indices

In [324]:
def flip_set(x,positions):
    y = x.copy()
    y[positions] = 1 - y[positions]
    return y

In [347]:
class EABJ:

    def __init__(self,n,lamb,min_dist,proba_pursue):
        self.n = n #nb iterations
        self.lamb = lamb #population size
        self.min_dist = min_dist
        self.proba_pursue = proba_pursue
    
    def __call__(self, problem: ioh.ProblemClass.GRAPH) -> None:
        dim = problem.meta_data.n_variables

        #Initialization
        x = np.random.randint(0,2,dim)
        y = problem(x)

        #Optimization
        for t in range(self.n-1):
            pop = []
            for i in range(self.lamb):
                l = positive_binomial(dim,3/dim,1)[0]
                I = select_distant_indices(dim, l, self.min_dist, self.proba_pursue)
                xx = flip_set(x,I)
                yy = problem(xx)
                pop.append((xx,yy))
            pop.sort(key=lambda x: x[1])
            if pop[self.lamb-1][1] >= y:
                x = pop[self.lamb-1][0]
                y = pop[self.lamb-1][1]

In [350]:
max_cut.reset()
dim = max_cut.meta_data.n_variables
eabj = EABJ(nb_it//10, 10, 4, 1/dim)
eabj(max_cut)
max_cut.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,

In [351]:
max_coverage.reset()
dim = max_coverage.meta_data.n_variables
eabj = EABJ(nb_it//10, 10, 5, 10/dim)
eabj(max_coverage)
max_coverage.state

<State evaluations: 9991 optimum_found: false current_best: <Solution x: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Bloc-note

Tester budget max et diminuer  
Descente de gradient stochastique // Relaxation  